In [1]:
#API + url init
import getpass
import os

API_KEY = ''
uri = ""


#genai.configure(api_key=API_KEY)

#from langchain_google_genai import GoogleGenerativeAIEmbeddings

google_api_key = API_KEY
os.environ["GOOGLE_API_KEY"] = google_api_key

#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#vector = embeddings.embed_documents(["hello, world!", "This is my second document" , "This is my third document"])







In [3]:
#Initialize variables + mongodb
import pymongo
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import ServiceContext

DB_NAME = "langchain_demo"
COLLECTION_NAME = 'collection_of_text_blobs'
INDEX_NAME = 'Indexx'


mongodb_client = pymongo.MongoClient(uri)
print ("Atlas client initialized")

embed_model = GeminiEmbedding(model_name="models/embedding-001")

vector = embed_model.get_text_embedding("Vector Search with MongoDB")
print(len(vector))


service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                 db_name = DB_NAME, collection_name = COLLECTION_NAME,
                                 index_name  = INDEX_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)


Atlas client initialized
768
LLM is explicitly disabled. Using MockLLM.


/tmp/ipykernel_16423/2111427378.py:19: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


In [32]:
#Embeds + uploads context files to mongoDB
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

data_dir = "./sample_files/"

docs = SimpleDirectoryReader(
        input_dir=data_dir
).load_data()

print (f"Loaded {len(docs)} chunks from '{data_dir}'")



index = VectorStoreIndex.from_documents(
    docs, 
    storage_context=storage_context,
    service_context=service_context,
)

Loaded 3 chunks from './sample_files/'


In [26]:
#Delete embedding files to add new context into the db
database = mongodb_client[DB_NAME]
collection = database[COLLECTION_NAME]

doc_count = collection.count_documents (filter = {})
print (f"Document count before delete : {doc_count:,}")

result = collection.delete_many(filter= {})
print (f"Deleted docs : {result.deleted_count}")

Document count before delete : 3
Deleted docs : 3


In [34]:
#Main driver Code 

import google.generativeai as genai
from IPython.display import Markdown, clear_output, display


query = "what are some problems with the pharmacy? Give me solutions to deal with it"


def generate_embedding(quer):
  temp = embed_model.get_text_embedding(quer)
  return temp

mongodb_client = pymongo.MongoClient(uri)
db = client.langchain_demo
collection = db.collection_of_text_blobs

results = collection.aggregate([
  {
    "$vectorSearch": {
      "queryVector": generate_embedding(query),
      "path": "embedding",
      "numCandidates": 50,
      "limit": 1,
      "index": "RAGIndexing",
    }
  }
])

context = ""
for document in results:
    print(type(document))
    print(f'Text present: {document["text"]}\n')
    context += document["text"]

print(context)

model=genai.GenerativeModel("gemini-pro")
#response = model.generate_content(query, stream=True)
response = model.generate_content([context, query], stream=True)

buffer = []
for chunk in response:
    for part in chunk.parts:
        buffer.append(part.text)
    clear_output()
    display(Markdown(''.join(buffer)))



**Problems with the Pharmacy:**

* **Slow customer service due to lack of electronic billing machine:** Leads to long wait times and frustrations for customers.
* **Unlabeled and disorganized medicine:** Makes it difficult to locate specific medications, wasting time and effort.

**Solutions:**

**Problem 1: Slow Customer Service**

* **Implement an electronic billing system:** Streamline the billing process, reducing wait times and improving customer satisfaction.
* **Train cashiers on efficient billing procedures:** Ensure cashiers are proficient in operating the electronic billing system to expedite transactions.
* **Consider self-service checkout kiosks:** Allow customers to scan and pay for their medications independently, freeing up cashiers for more complex tasks.

**Problem 2: Unlabeled and Disorganized Medicine**

* **Create a clear and logical labeling system:** Use color-coding, alphabetical order, or other methods to categorize and identify medications.
* **Implement a medication inventory management system:** Track medication inventory levels and optimize storage to ensure supplies are readily available.
* **Arrange medications by category or prescription:** Make it easier for customers and staff to locate specific medications quickly and efficiently.
* **Provide clear instructions on medication storage:** Ensure medications are stored in a safe and accessible manner.

Updating mongoDB so that the embeddings are indexed
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "dimensions": 768,
        "similarity": "dotProduct",
        "type": "knnVector"
      }
    }
  }
}

In [ ]:
"""
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

llm = Gemini(model="models/gemini-pro")
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)
query_llm = index.as_query_engine()
print(query_llm.query("What does this report outline?"))
"""
# Import the necessary libararies
import os
from pymongo import MongoClient
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex



mongodb_client = MongoClient(uri)


embed_model = GeminiEmbedding(model_name="models/embedding-001")

llm = Gemini(model="models/gemini-pro")

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                 db_name = DB_NAME, collection_name = COLLECTION_NAME,
                                 index_name  = INDEX_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

query_llm = index.as_query_engine()

            
prompt = "what does Complicated dosing schedules lead to?"

answer = query_llm.query(prompt)

print(answer)


In [ ]:
#LoadData 

from pymongo import MongoClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import MongoDBAtlasVectorSearch 

#from langchain.llms import GoogleGenerativeAIEmbeddings
#from langchain.llms import gemini
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import google.generativeai as genai
from langchain.document_loaders import TextLoader


#Initializing the database(Files samplespace)
client = MongoClient(uri)
dbName = "langchain_demo"
collectionName = "collection_of_text_blobs"
collectionn = client[dbName][collectionName]

#dataloader from directory
#loader = DirectoryLoader( './sample_files/', glob = "./*txt", show_progress = True)
loader = TextLoader("./sample_files/sample1.txt")
data = loader.load()
print(data)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#vectorStore = MongoDBAtlasVectorSearch.from_documents(data, embeddings, collection=collectionn)
vector = embeddings.embed_documents(data)
print(vector)


#embeddings_list = list(embeddings)
#embeddings_dict = [embedding[0].to_dict() for embedding in embeddings_list]



#vectorStore = MongoDBAtlasVectorSearch.from_documents(data, embeddings)



In [ ]:

model = genai.GenerativeModel(model_name='gemini-pro')
response = model.generate_content(prompt, stream=True)
embeddings = GoogleGenerativeAIEmbeddings

google.generativeai.embed_content(
    model: "models/text-embedding-004" ,
    content: (content_types.ContentType | Iterable[content_types.ContentType]),
    task_type: (EmbeddingTaskTypeOptions | None) = None,
    title: (str | None) = None,
    output_dimensionality: (int | None) = None,
    client: glm.GenerativeServiceClient = None,
    request_options: (dict[str, Any] | None) = None
) -> (text_types.EmbeddingDict | text_types.BatchEmbeddingDict)